In [1]:
import pynq
import dishsoap
import numpy as np

dish = dishsoap.Overlay()

In [2]:
results = pynq.allocate(shape=(1001,),dtype=np.uint64)

In [3]:
%time dish.run_synch_inplace([1,1,1], results)

CPU times: user 1.52 ms, sys: 136 µs, total: 1.66 ms
Wall time: 1.92 ms


In [4]:
dish.unpack(results)

array([[1, 1, 1, 0, 0],
       [1, 1, 0, 1, 0],
       [1, 1, 0, 0, 1],
       ...,
       [1, 1, 0, 0, 0],
       [1, 1, 0, 0, 0],
       [1, 1, 0, 0, 0]], dtype=uint8)

In [5]:
results.shape

(1001,)

In [6]:
# generate all network states
all_init_states = np.arange(2**dish.network_size, dtype=np.uint64)

for init in all_init_states:
    # pretty-print the network state
    print('init state:', dish.unpack(init))
    # run each network state for some number of iterations
    results = dish.run_synch_np(init, 5)
    result_array = np.array([dish.unpack(r) for r in results])
    print(result_array)

init state: [0 0 0 0 0]
[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
init state: [1 0 0 0 0]
[[1 0 0 0 0]
 [1 0 1 0 0]
 [1 0 1 1 0]
 [1 0 1 1 1]
 [1 0 1 1 1]
 [1 0 1 1 1]]
init state: [0 1 0 0 0]
[[0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]]
init state: [1 1 0 0 0]
[[1 1 0 0 0]
 [1 1 0 0 0]
 [1 1 0 0 0]
 [1 1 0 0 0]
 [1 1 0 0 0]
 [1 1 0 0 0]]
init state: [0 0 1 0 0]
[[0 0 1 0 0]
 [0 0 0 1 0]
 [0 0 0 0 1]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
init state: [1 0 1 0 0]
[[1 0 1 0 0]
 [1 0 1 1 0]
 [1 0 1 1 1]
 [1 0 1 1 1]
 [1 0 1 1 1]
 [1 0 1 1 1]]
init state: [0 1 1 0 0]
[[0 1 1 0 0]
 [0 1 0 1 0]
 [0 1 0 0 1]
 [0 1 0 0 0]
 [0 1 0 0 0]
 [0 1 0 0 0]]
init state: [1 1 1 0 0]
[[1 1 1 0 0]
 [1 1 0 1 0]
 [1 1 0 0 1]
 [1 1 0 0 0]
 [1 1 0 0 0]
 [1 1 0 0 0]]
init state: [0 0 0 1 0]
[[0 0 0 1 0]
 [0 0 0 0 1]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
init state: [1 0 0 1 0]
[[1 0 0 1 0]
 [1 0 1 0 1]
 [1 0 1 1 0]
 [1 0 1 1 

In [7]:
init_state = dishsoap.list_to_packed([1,1,0])
results = pynq.allocate(shape=(2000,), dtype=np.uint64)
%time dish.run_synch_inplace(init_state, results)
print('number states saved:', len(results))
print('first two results:\r\n', dish.unpack(results[:2]))
print('last two results:\r\n',  dish.unpack(results[-2:]))

CPU times: user 819 µs, sys: 79 µs, total: 898 µs
Wall time: 951 µs
number states saved: 2000
first two results:
 [[1 1 0 0 0]
 [1 1 0 0 0]]
last two results:
 [[1 1 0 0 0]
 [1 1 0 0 0]]


In [8]:
%%time 
def benchmark(init_states, iterations = 5000):
    print('---allocating all memory---')
    n_states = len(init_states)
    %time pynq.allocate(shape=(iterations*n_states,), dtype=np.uint64)
    results = pynq.allocate(shape=(iterations,n_states), dtype=np.uint64)
    print('---running DiSH---')
    for s, r in zip(init_states, results):
        %time dish.run_synch_inplace(s, r)
    print('---total time---')
    # show first and last, prove we did it all
    #print(dish.unpack(results[:n_states,:1]))
    #print(dish.unpack(results[:n_states,-1:]))
    
benchmark(all_init_states)

---allocating all memory---
CPU times: user 19.6 ms, sys: 10.5 ms, total: 30.1 ms
Wall time: 40.5 ms
---running DiSH---
CPU times: user 2.47 ms, sys: 0 ns, total: 2.47 ms
Wall time: 2.32 ms
CPU times: user 3.34 ms, sys: 0 ns, total: 3.34 ms
Wall time: 3.06 ms
CPU times: user 4.02 ms, sys: 0 ns, total: 4.02 ms
Wall time: 3.85 ms
CPU times: user 1.06 ms, sys: 0 ns, total: 1.06 ms
Wall time: 986 µs
CPU times: user 846 µs, sys: 0 ns, total: 846 µs
Wall time: 900 µs
CPU times: user 949 µs, sys: 0 ns, total: 949 µs
Wall time: 1 ms
CPU times: user 887 µs, sys: 0 ns, total: 887 µs
Wall time: 945 µs
CPU times: user 2.74 ms, sys: 69 µs, total: 2.81 ms
Wall time: 2.6 ms
CPU times: user 3.4 ms, sys: 0 ns, total: 3.4 ms
Wall time: 3.09 ms
CPU times: user 3.46 ms, sys: 0 ns, total: 3.46 ms
Wall time: 3.16 ms
CPU times: user 3.45 ms, sys: 0 ns, total: 3.45 ms
Wall time: 3.19 ms
CPU times: user 3.41 ms, sys: 0 ns, total: 3.41 ms
Wall time: 3.11 ms
CPU times: user 3.45 ms, sys: 0 ns, total: 3.45 ms
Wal